I'm starting off by cpying the code from my Basics Badge Project to setup a DataFrame from the .json file... before moving on to trying out other vecorization and/or prediciton strategies.

In [1]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
#from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import pandas as pd
#import os
#import re
from string import punctuation
#from datetime import datetime

In [2]:
# may need to download stopwords from nltk

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cman0\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cman0\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cman0\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\cman0\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
# define list of stopwords
english_stopwords = stopwords.words('english')
english_stopwords += punctuation

# create function to clean the question bank
def clean_wordlist(questions):
    '''
    Takes a list of questions and cleans them into a tokenized list
    of lemmatized words.
    '''
    corpus = ''
    wordlist = []
    lemmatizer = WordNetLemmatizer()
    for q in questions:
        corpus += q.lower()
    wordlist += [lemmatizer.lemmatize(word) for word in
                word_tokenize(corpus) if word not in english_stopwords]
    return wordlist

In [4]:
# create df from the .json file
df = pd.read_json('jeopardy.json')

# turn $values into int's
df['value'] = df['value'].str.replace('[^0-9]', '', regex=True)
df['value'] = df['value'].fillna(0)
df['value'] = df['value'].astype('int')

# create new collumn in df containing binary values for high vs low
df['high_low'] = [1 if x>600 else 0 for x in df['value']]

# create new collumn in df containing cleaned questions.

# NO LONGER NEEDED
#df['q_words'] = [' '.join(clean_wordlist([x])) for x in df['question']]

In [5]:
df

,category,air_date,question,value,answer,round,show_number,high_low
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",200,Copernicus,Jeopardy!,4680,0
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,200,Jim Thorpe,Jeopardy!,4680,0
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,200,Arizona,Jeopardy!,4680,0
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",200,McDonald\'s,Jeopardy!,4680,0
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",200,John Adams,Jeopardy!,4680,0
...,...,...,...,...,...,...,...,...
216925,RIDDLE ME THIS,2006-05-11,'This Puccini opera turns on the solution to 3...,2000,Turandot,Double Jeopardy!,4999,1
216926,"""T"" BIRDS",2006-05-11,'In North America this term is properly applie...,2000,a titmouse,Double Jeopardy!,4999,1
216927,AUTHORS IN THEIR YOUTH,2006-05-11,"'In Penny Lane, where this ""Hellraiser"" grew u...",2000,Clive Barker,Double Jeopardy!,4999,1
216928,QUOTATIONS,2006-05-11,"'From Ft. Sill, Okla. he made the plea, Arizon...",2000,Geronimo,Double Jeopardy!,4999,1


Ok, back to where we were on the Basics Badge right before training our model using tfidf and naive bayes. This time I plan to use word embeddings from a pre-trained word2vec model. I guess the first thing to do is download the the GoogleNews vectors we'll be using.

In [2]:
# download GoogleNews vectors file if needed
# !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2022-02-13 19:09:41--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.139.109
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.139.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G   170KB/s    in 2h 56m  

2022-02-13 22:06:34 (152 KB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



That took a while.... ok now to grab the keyed vectors from the trained model.

In [6]:
import gensim

In [7]:
embeddings_file = './GoogleNews-vectors-negative300.bin'
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(embeddings_file, binary=True)

I tried making this next part a function, but... it didn't work out. Just gonna do it like this and comment along the way.

Basically running each word (of each question) through Google's word embeddings to get their word vector arrays, and then getting the averaging (mean) of the word vector arrays for each question, which gives us a "question vector" array.

In [37]:
# googled a way to stop the "FutureWarning" message from coming up for every itteration
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#hope it works because that was really annoying

question_vectors = pd.DataFrame()
    # creating a string of lowercase words for each questuion and 
    # itterating through them
for question in df['question'].str.lower().str.replace('[^a-z ]',''):
    # creating a temporary datafame to hold all of the word vectors
    temp = pd.DataFrame()
    # itterating through each word
    for word in question.split(' '):
        # skip stop words
        if not word in english_stopwords:
                   # skip words not in Google's word embeddings
            try:   # grab the vector for the word
                word_vec = word_vectors[word]
                # put it in the temp dataframe
                temp = temp.append(pd.Series(word_vec), ignore_index=True)
            except:
                pass
        # create vector for this question using the average
        # of the vectors of all the words in the question
    this_q_vec = temp.mean()
        # put the question's vector into the main DataFrame
    question_vectors = question_vectors.append(this_q_vec, ignore_index=True)

Got lots of warnings there!
FutureWarning? ...sounds like a problem for another day. :D
Guess I'll eventuall have to learn how to use pandas.concat instead of the frame.append method.

But today's problem is that the VM I'm on can't handle this. I guess we've outgrown this. Time to upgrade.

In [38]:
print(question_vectors.shape)
print(type(question_vectors))
question_vectors

(216930, 300)
<class 'pandas.core.frame.DataFrame'>


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.053513,0.028366,-0.003403,0.060440,-0.011749,0.018772,0.052729,-0.096954,0.073669,0.170628,...,-0.116028,0.010610,-0.076401,0.055969,-0.106514,-0.060791,-0.023376,-0.097069,0.030815,0.033997
1,-0.070263,0.130393,0.040058,0.083141,0.067061,-0.009216,-0.016324,-0.120292,-0.052221,0.085904,...,0.007724,-0.054294,-0.065976,-0.066806,0.032055,-0.100475,-0.066428,-0.057850,0.082839,0.022688
2,-0.009086,0.069615,0.034862,0.038376,0.028006,-0.093994,-0.009918,-0.081107,0.069406,0.085458,...,-0.020455,-0.007708,-0.079356,-0.022810,0.028102,-0.041992,-0.089992,-0.023980,0.044006,-0.089634
3,-0.037720,-0.066895,-0.001670,0.121809,-0.046596,0.005515,0.122977,-0.070626,0.034180,0.102086,...,-0.056553,-0.096819,-0.239641,0.132499,0.092111,0.002973,-0.082136,-0.019723,-0.042411,-0.167899
4,0.068912,-0.043349,0.059557,0.100227,-0.057617,-0.041477,-0.008450,-0.088745,0.126736,0.021566,...,-0.074632,-0.128906,-0.114217,0.062384,-0.047323,-0.034776,-0.146484,-0.012926,0.143745,0.067464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216925,0.107361,-0.002075,-0.014933,0.108297,-0.053955,0.021301,0.122599,0.017049,0.013590,0.141591,...,-0.253113,0.040649,-0.240438,-0.003825,-0.082634,0.094910,-0.000905,0.041829,0.086100,0.024526
216926,-0.091732,0.001933,-0.048096,0.088677,-0.008965,-0.138165,-0.011814,-0.078559,0.034383,0.077260,...,-0.054901,0.001689,-0.077101,0.111030,0.054688,-0.004869,0.003432,-0.007151,0.124410,-0.069255
216927,0.017008,0.138156,0.094830,0.047065,-0.004895,0.093838,-0.060109,-0.115553,0.040952,0.088121,...,0.038896,-0.038656,-0.206906,0.078179,-0.065199,-0.045064,-0.102254,0.066569,0.072320,0.055420
216928,-0.009044,0.027582,0.027116,0.057123,-0.017256,-0.122869,0.044478,-0.030525,0.089511,0.046853,...,-0.144798,0.059027,-0.053511,-0.043599,0.018205,0.007990,-0.062600,-0.144731,0.088224,-0.081587


Yay! My husband's gaming PC was able to handle it. Moving on...

In [40]:
question_vectors['value'] = df['high_low']
question_vectors

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,value
0,0.053513,0.028366,-0.003403,0.060440,-0.011749,0.018772,0.052729,-0.096954,0.073669,0.170628,...,0.010610,-0.076401,0.055969,-0.106514,-0.060791,-0.023376,-0.097069,0.030815,0.033997,0
1,-0.070263,0.130393,0.040058,0.083141,0.067061,-0.009216,-0.016324,-0.120292,-0.052221,0.085904,...,-0.054294,-0.065976,-0.066806,0.032055,-0.100475,-0.066428,-0.057850,0.082839,0.022688,0
2,-0.009086,0.069615,0.034862,0.038376,0.028006,-0.093994,-0.009918,-0.081107,0.069406,0.085458,...,-0.007708,-0.079356,-0.022810,0.028102,-0.041992,-0.089992,-0.023980,0.044006,-0.089634,0
3,-0.037720,-0.066895,-0.001670,0.121809,-0.046596,0.005515,0.122977,-0.070626,0.034180,0.102086,...,-0.096819,-0.239641,0.132499,0.092111,0.002973,-0.082136,-0.019723,-0.042411,-0.167899,0
4,0.068912,-0.043349,0.059557,0.100227,-0.057617,-0.041477,-0.008450,-0.088745,0.126736,0.021566,...,-0.128906,-0.114217,0.062384,-0.047323,-0.034776,-0.146484,-0.012926,0.143745,0.067464,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216925,0.107361,-0.002075,-0.014933,0.108297,-0.053955,0.021301,0.122599,0.017049,0.013590,0.141591,...,0.040649,-0.240438,-0.003825,-0.082634,0.094910,-0.000905,0.041829,0.086100,0.024526,1
216926,-0.091732,0.001933,-0.048096,0.088677,-0.008965,-0.138165,-0.011814,-0.078559,0.034383,0.077260,...,0.001689,-0.077101,0.111030,0.054688,-0.004869,0.003432,-0.007151,0.124410,-0.069255,1
216927,0.017008,0.138156,0.094830,0.047065,-0.004895,0.093838,-0.060109,-0.115553,0.040952,0.088121,...,-0.038656,-0.206906,0.078179,-0.065199,-0.045064,-0.102254,0.066569,0.072320,0.055420,1
216928,-0.009044,0.027582,0.027116,0.057123,-0.017256,-0.122869,0.044478,-0.030525,0.089511,0.046853,...,0.059027,-0.053511,-0.043599,0.018205,0.007990,-0.062600,-0.144731,0.088224,-0.081587,1


Got an error in a later step saying that the data contains null values. I guess there were a few questions that didn't contain any words from GoogleNews. Removing all lines with null values.

In [44]:
question_vectors = question_vectors.dropna()
question_vectors

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,value
0,0.053513,0.028366,-0.003403,0.060440,-0.011749,0.018772,0.052729,-0.096954,0.073669,0.170628,...,0.010610,-0.076401,0.055969,-0.106514,-0.060791,-0.023376,-0.097069,0.030815,0.033997,0
1,-0.070263,0.130393,0.040058,0.083141,0.067061,-0.009216,-0.016324,-0.120292,-0.052221,0.085904,...,-0.054294,-0.065976,-0.066806,0.032055,-0.100475,-0.066428,-0.057850,0.082839,0.022688,0
2,-0.009086,0.069615,0.034862,0.038376,0.028006,-0.093994,-0.009918,-0.081107,0.069406,0.085458,...,-0.007708,-0.079356,-0.022810,0.028102,-0.041992,-0.089992,-0.023980,0.044006,-0.089634,0
3,-0.037720,-0.066895,-0.001670,0.121809,-0.046596,0.005515,0.122977,-0.070626,0.034180,0.102086,...,-0.096819,-0.239641,0.132499,0.092111,0.002973,-0.082136,-0.019723,-0.042411,-0.167899,0
4,0.068912,-0.043349,0.059557,0.100227,-0.057617,-0.041477,-0.008450,-0.088745,0.126736,0.021566,...,-0.128906,-0.114217,0.062384,-0.047323,-0.034776,-0.146484,-0.012926,0.143745,0.067464,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216925,0.107361,-0.002075,-0.014933,0.108297,-0.053955,0.021301,0.122599,0.017049,0.013590,0.141591,...,0.040649,-0.240438,-0.003825,-0.082634,0.094910,-0.000905,0.041829,0.086100,0.024526,1
216926,-0.091732,0.001933,-0.048096,0.088677,-0.008965,-0.138165,-0.011814,-0.078559,0.034383,0.077260,...,0.001689,-0.077101,0.111030,0.054688,-0.004869,0.003432,-0.007151,0.124410,-0.069255,1
216927,0.017008,0.138156,0.094830,0.047065,-0.004895,0.093838,-0.060109,-0.115553,0.040952,0.088121,...,-0.038656,-0.206906,0.078179,-0.065199,-0.045064,-0.102254,0.066569,0.072320,0.055420,1
216928,-0.009044,0.027582,0.027116,0.057123,-0.017256,-0.122869,0.044478,-0.030525,0.089511,0.046853,...,0.059027,-0.053511,-0.043599,0.018205,0.007990,-0.062600,-0.144731,0.088224,-0.081587,1


Now to split the data into train and test subsets.

In [54]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(question_vectors.drop('value', axis=1),
                                                   question_vectors['value'],
                                                   test_size = 0.2,
                                                   random_state = 1)
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((172156, 300), (43039, 300), (172156,), (43039,))

Multiple examples of similar code I found used AdaBoostClassifier from scikit-learn for the next part, so gonna give that a try and use here instead of Naive Bayes.

In [46]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

AdaBoost = AdaBoostClassifier(random_state = 1)
AdaBoost.fit(train_x, train_y)
test_pred = AdaBoost.predict(test_x)


print('Accuracy: {:.2f}'.format(accuracy_score(test_y, test_pred)))

Accuracy: 0.57


Wow! Not great. I really expected it to be higher. Wonder if there's anything that could be done to improve it...

Found a neat flowchart about different selecting the right machince learning algorithm based off the data you have and what you're trying to do with it.

https://scikit-learn.org/stable/_static/ml_map.png

It suggests that SGD Classifier is proabably best for us. Let's give it a try.

In [47]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="log", penalty="l2")
clf.fit(train_x, train_y)
y_pred = clf.predict(test_x)

print('Accuracy: {:.2f}'.format(accuracy_score(test_y, y_pred)))

Accuracy: 0.57


Well, that wan't great, either. Ok, supposedly scaling is important when using SGD Classifier. I'll try that.

In [50]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier

scaler = StandardScaler()
scaler.fit(train_x)
train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

clf = SGDClassifier(loss="log", penalty="l2")
clf.fit(train_x, train_y)
y_pred = clf.predict(test_x)

print('Accuracy: {:.2f}'.format(accuracy_score(test_y, y_pred)))

Accuracy: 0.55


not better. maybe go back to Naive_Bayes??? but with the vectors from google.

In [55]:
from sklearn.naive_bayes import MultinomialNB

naive_bayes = MultinomialNB()
naive_bayes.fit(train_x, train_y)
y_pred = naive_bayes.predict(test_x)
print('Accuracy: {:.2f}'.format(accuracy_score(test_y, y_pred)))

ValueError: Negative values in data passed to MultinomialNB (input X)

I guess these types of vectors are not meant to be used with Naive Bayes. I'll try one or two more form the list before calling it quits.

In [56]:
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier()
classifier.fit(train_x, train_y)
y_pred = classifier.predict(test_x)

print('Accuracy: {:.2f}'.format(accuracy_score(test_y, y_pred)))

Accuracy: 0.53


In [57]:
from sklearn.svm import LinearSVC

classifier = LinearSVC()
classifier.fit(train_x, train_y)
y_pred = classifier.predict(test_x)

print('Accuracy: {:.2f}'.format(accuracy_score(test_y, y_pred)))

Accuracy: 0.57


Don't know what else to try to get that number higher. I give up.